# REINFORCE 与 A2C 原理笔记
## 一、策略梯度方法基础
策略梯度方法的核心思想是将策略从表格形式转化为函数形式（如神经网络），通过优化参数化策略以最大化目标函数。其核心流程为：定义目标函数→计算目标函数梯度→通过梯度上升优化策略。
1. 目标函数
策略梯度方法的目标函数用于衡量策略性能，主要包括两类：
平均状态值（
v
  
π
​
 
）
定义为状态值的加权平均：
v
  
π
​
 =∑ 
s∈S
​
 d(s)v 
π
​
 (s)=E 
S∼d
​
 [v 
π
​
 (S)]
其中，
d(s)
 是状态权重（概率分布），
v 
π
​
 (s)
 是状态
s
的长期回报期望。
若
d
独立于策略（记为
d 
0
​
 
），可设为均匀分布（
d 
0
​
 (s)=1/∣S∣
）或仅关注起始状态（
d 
0
​
 (s 
0
​
 )=1
）。
若
d
依赖策略（记为
d 
π
​
 
），通常采用策略
π
的平稳分布（长期访问频率越高的状态权重越大）。
平均奖励（
r
  
π
​
 
）
定义为单步即时奖励的加权平均：
r
  
π
​
 =∑ 
s∈S
​
 d 
π
​
 (s)r 
π
​
 (s)=E 
S∼d 
π
​
 
​
 [r 
π
​
 (S)]
其中，
r 
π
​
 (s)=∑ 
a∈A
​
 π(a∣s)r(s,a)
 是状态
s
下的平均单步奖励。
其等价形式为无穷步奖励的平均极限：
r
  
π
​
 =lim 
n→∞
​
  
n
1
​
 E[∑ 
k=1
n
​
 R 
t+k
​
 ∣S 
t
​
 =s 
0
​
 ]
两者关系：在折扣场景（
γ∈(0,1)
）中，
r
  
π
​
 =(1−γ) 
v
  
π
​
 
，优化其一等价于优化其二。
2. 目标函数梯度
目标函数梯度是策略更新的核心，其通用形式为：
∇ 
θ
​
 J(θ)=E[∇ 
θ
​
 lnπ(A∣S,θ)⋅q 
π
​
 (S,A)]
其中：
J(θ)
 可为
v
  
π
​
 
、
r
  
π
​
 
等目标函数；
π(a∣s,θ)
 是参数化策略（如神经网络输出的动作概率）；
q 
π
​
 (s,a)
 是动作值函数（状态
s
下采取动作
a
的长期回报期望）；
推导核心：利用
∇ 
θ
​
 π(a∣s,θ)=π(a∣s,θ)⋅∇ 
θ
​
 lnπ(a∣s,θ)
，将梯度转化为对数概率的期望形式，便于采样估计。
3. 策略表示与探索利用平衡
策略表示：通常用 softmax 函数确保输出为合法概率分布：
π(a∣s,θ)= 
∑ 
a 
′
 ∈A
​
 e 
h(s,a 
′
 ,θ)
 
e 
h(s,a,θ)
 
​
 
其中
h(s,a,θ)
为神经网络输出的分数，softmax 保证
π(a∣s,θ)∈(0,1)
且和为 1。
探索与利用：通过参数更新自动平衡：
利用（Exploitation）：若
q 
π
​
 (s,a)
较大（动作价值高），参数更新会提高
π(a∣s,θ)
；
探索（Exploration）：若
π(a∣s,θ)
较小（动作被选概率低），参数更新会增强其概率。
## 二、REINFORCE 算法
REINFORCE 是最早的策略梯度算法之一，基于蒙特卡洛（Monte Carlo）方法估计动作值函数，属于离线（offline）算法。
1. 核心思想
用蒙特卡洛估计的回报（Gt） 近似动作值函数
q 
π
​
 (s,a)
，再通过梯度上升更新策略参数。
2. 参数更新公式
θ 
t+1
​
 =θ 
t
​
 +α⋅∇ 
θ
​
 lnπ(a 
t
​
 ∣s 
t
​
 ,θ 
t
​
 )⋅G 
t
​
 
其中：
α
 是学习率；
G 
t
​
 =∑ 
k=0
T−t
​
 γ 
k
 R 
t+k+1
​
 
 是从时刻
t
开始的折扣回报（蒙特卡洛估计的动作值）。
3. 算法流程（伪代码）
初始化策略参数
θ
、学习率
α
、折扣因子
γ
；
对于第
k
次迭代：
从初始状态
s 
0
​
 
出发，根据当前策略
π(⋅∣⋅,θ)
生成完整轨迹：
(s 
0
​
 ,a 
0
​
 ,R 
1
​
 ,s 
1
​
 ,a 
1
​
 ,R 
2
​
 ,...,s 
T−1
​
 ,a 
T−1
​
 ,R 
T
​
 )
；
对轨迹中每个时刻
t
（
0≤t<T
）：
计算回报
G 
t
​
 =R 
t+1
​
 +γR 
t+2
​
 +...+γ 
T−t−1
 R 
T
​
 
；
更新参数：
θ=θ+α⋅∇ 
θ
​
 lnπ(a 
t
​
 ∣s 
t
​
 ,θ)⋅G 
t
​
 
；
重复迭代直至收敛。
4. 特点
优点：实现简单，直接优化策略，无需构建复杂价值函数，适用于连续动作空间；
缺点：
方差大（依赖完整轨迹的回报估计）；
离线更新（需等待轨迹结束才能更新参数），样本利用率低；
对学习率敏感，收敛不稳定。
## 三、A2C（Advantage Actor-Critic）算法
A2C 是 actor-critic 框架的典型代表，结合了策略梯度（actor）和价值函数估计（critic），属于在线（online）算法，可减少方差并提高效率。
1. 核心思想
Actor：负责策略优化，输出动作概率
π(a∣s,θ)
，类似 REINFORCE；
Critic：负责估计优势函数（Advantage Function） 
A(s,a)=q 
π
​
 (s,a)−v 
π
​
 (s)
，用于指导 actor 更新，减少方差。
2. 优势函数
优势函数衡量动作
a
相对于状态
s
平均价值的优劣：
若
A(s,a)>0
：动作
a
优于状态
s
的平均动作；
若
A(s,a)<0
：动作
a
劣于平均。
通过时序差分（TD）方法估计：
A(s 
t
​
 ,a 
t
​
 )≈R 
t+1
​
 +γv(s 
t+1
​
 )−v(s 
t
​
 )
，其中
v(s)
是状态值函数（critic 输出）。
3. 参数更新公式
Actor 更新（策略梯度）：
θ 
t+1
​
 =θ 
t
​
 +α⋅∇ 
θ
​
 lnπ(a 
t
​
 ∣s 
t
​
 ,θ 
t
​
 )⋅A(s 
t
​
 ,a 
t
​
 )
Critic 更新（价值函数拟合）：
通过 TD 误差优化
v(s)
：
v(s 
t
​
 )←v(s 
t
​
 )+β⋅(R 
t+1
​
 +γv(s 
t+1
​
 )−v(s 
t
​
 ))
，其中
β
是 critic 的学习率。
4. 算法流程
初始化 actor 参数
θ
、critic 参数
ϕ
（用于估计
v(s;ϕ)
）、学习率
α,β
、折扣因子
γ
；
对于每个时间步
t
：
Actor 根据
π(⋅∣s 
t
​
 ,θ)
选择动作
a 
t
​
 
，与环境交互得到
R 
t+1
​
 ,s 
t+1
​
 
；
Critic 计算 TD 误差：
δ 
t
​
 =R 
t+1
​
 +γv(s 
t+1
​
 ;ϕ)−v(s 
t
​
 ;ϕ)
（即优势函数近似
A(s 
t
​
 ,a 
t
​
 )
）；
更新 actor：
θ=θ+α⋅∇ 
θ
​
 lnπ(a 
t
​
 ∣s 
t
​
 ,θ)⋅δ 
t
​
 
；
更新 critic：
ϕ=ϕ+β⋅δ 
t
​
 ⋅∇ 
ϕ
​
 v(s 
t
​
 ;ϕ)
；
重复迭代直至收敛。
5. 特点
优点：
方差小（用优势函数替代蒙特卡洛回报，减少噪声）；
在线更新（每步即可更新参数），样本利用率高；
收敛更稳定，适合复杂环境；
缺点：需同时训练 actor 和 critic，参数调优更复杂；critic 的估计误差可能影响 actor 更新。
## 四、REINFORCE 与 A2C 的对比
维度	REINFORCE	A2C
价值估计方法	蒙特卡洛（完整轨迹回报）	时序差分（TD 误差，优势函数）
更新方式	离线（轨迹结束后更新）	在线（每步更新）
方差	高	低（优势函数减少噪声）
样本利用率	低	高
复杂度	简单（仅需训练 actor）	复杂（需同步训练 actor 和 critic）
适用场景	简单环境，对实时性要求低	复杂环境，实时性要求